# Exploring Twitter Data with Vector Databases and RAG Systems

This tutorial introduces the creation of a vector database and the use of a retrieval-augmented generation (RAG) system to explore Twitter data interactively. 

Please check [LBSocial](www.lbsocial.net)  on how to collect Twitter data. 

## Set up a Database and API Keys

Create a [MongoDB](www.mongodb.com) cluster and store the connection string in a safe place, such as AWS Secrets Manager. 
- key name: `connection_string`
- key value: <`the connection string`>, you need to type the password
- secret name: `mongodb`


You also need to purchase and your [oepnai](https://openai.com/) api key in AWS Secrets Manager:
- key name: `api_key`
- key value: <`your openai api key`>
- secret name: `openai`

## Install Python Libraries

- pymongo: manage the MongoDB database
- openai: create embeddings and resonpses 

In [1]:
pip install pymongo openai -q

Note: you may need to restart the kernel to use updated packages.


## Secrets Manager Function

In [2]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Import Python Libraries and Credentials

In [3]:
import pymongo
from pymongo import MongoClient
import json
import re
import os
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)


mongodb_connect = get_secret('mongodb')['connection_string']


## Connect to the MongoDB cluster

In [4]:
mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection

## Utility Funcitons

- the `clean_tweet` function removes URLs in tweets
- the `get_embedding` function use openai to create tweet embeddings
- the `vector_search` function return relevent tweets based on a query

In [5]:
def clean_tweet(text):
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(url_pattern, '', text)

In [6]:
embedding_model= 'text-embedding-3-small'

def get_embedding(text):

    try:
        embedding = client.embeddings.create(input=text, model=embedding_model).data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

In [7]:
def vector_search(query):

    query_embedding = get_embedding(query)
    if query_embedding is None:
        return "Invalid query or embedding generation failed."
    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "tweet_vector",
                "queryVector": query_embedding,
                "path": "tweet.embedding",
                "numCandidates": 1000,  # Number of candidate matches to consider
                "limit": 10  # Return top 10 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "tweet.text": 1 # return tweet text

            }
        }
    ]

    results = tweet_collection.aggregate(pipeline)
    return list(results)

## Tweets Embedding 

For more about text embeddings please read [Introducing text and code embeddings](https://openai.com/index/introducing-text-and-code-embeddings/)

In [8]:
from tqdm.auto import tqdm
tweets = tweet_collection.find()

for tweet in tqdm(list(tweets)):
    try:
        tweet_embedding = get_embedding(clean_tweet(tweet['tweet']['text']))
    #     print(tweet_embedding)

        tweet_collection.update_one(
            {'tweet.id':tweet['tweet']['id']},
            {"$set":{'tweet.embedding':tweet_embedding}}
        )
    except:
        print(f"""error in embedding tweet {tweet['tweet']['id']}""")
        pass


  0%|          | 0/269 [00:00<?, ?it/s]

## Create a Vector Index

For more about the MognoDB Vector database, please read [What are Vector Databases?](https://www.mongodb.com/resources/basics/databases/vector-databases)
This code creates a vector index following the [MongoDB official document](https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#std-label-avs-types-vector-search).

In [9]:
# Create your index model, then create the search index

from pymongo.operations import SearchIndexModel
import time

search_index_model = SearchIndexModel(
  definition={
  "fields": [
    {
      "type": "vector",
      "path": "tweet.embedding",
      "numDimensions": 1536,
      "similarity": "cosine"
    }
  ]
},
  name="tweet_vector",
  type="vectorSearch"

)
result = tweet_collection.create_search_index(model=search_index_model)
print("New search index named " + result + " is building.")
# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")
predicate=None

if predicate is None:
  predicate = lambda index: index.get("queryable") is True

while True:
  indices = list(tweet_collection.list_search_indexes(result))
  if len(indices) and predicate(indices[0]):

    break
  time.sleep(5)

print(result + " is ready for querying.")

New search index named tweet_vector is building.
Polling to check if the index is ready. This may take up to a minute.
tweet_vector is ready for querying.


In [13]:
user_query = 'clothes'

for tweet in vector_search(user_query):
    print(tweet['tweet']['text'])

hugyf
Kǝnǝngadǝ andega lejin

⎐ٺون⎐
⊵SAAS⊴

⎐شي▬ان⎐
⊵artm15⊴

⎐مغربي▬للبصريات⎐
⊵T5⊴

⎐بٱث▬ٱند.بؤدى⎐
⊵A4TU⊴
jMU
hugyf
Kǝnǝngadǝ andega lejin

⎐ٺون⎐
⊵SAAS⊴

⎐شي▬ان⎐
⊵artm15⊴

⎐مغربي▬للبصريات⎐
⊵T5⊴

⎐بٱث▬ٱند.بؤدى⎐
⊵A4TU⊴
JmU
Sai shiye waltǝyi

⎐بلؤمنغدَىلز▬بلؤمينغدَىلز⎐
⊵X59⊴

⎐ٺوٺ⎐
⊵SAAS⊴

⎐شي▬ان⎐
⊵artm15⊴

⎐بوترى▬بارن⎐
⊵yfm4⊴
JmU
RT @akijikure: 今日は運動がてら
中央桟橋〜からすこまで歩いた😅
JMU敷地内一部伐採してるから
第4ドックも真正面スッキリ
してました。 https://t.co/xlnoaTnfzl
Ya Kǝma ilmunǝmbe

⎐ٺون⎐
⊵SAAS⊴

⎐ماماز▬اند▬باباز⎐
⊵DF60⊴

⎐شي▬ان⎐
⊵artm15⊴

⎐مفارش▬الحبىب⎐
⊵C51⊴
JMU
Ya Kǝma ilmunǝmbe

⎐ٺون⎐
⊵SAAS⊴

⎐ماماز▬اند▬باباز⎐
⊵DF60⊴

⎐شي▬ان⎐
⊵artm15⊴

⎐مفارش▬الحبىب⎐
⊵C51⊴
JMu
Ilmu kimiyabe galiwudǝ

⎐نمشؾ⎐
⊵CMM⊴

⎐بٱث⎐
⊵A4TU⊴

⎐ٺوٺ⎐
⊵SAAS⊴
jmu
Sai shiye waltǝyiShim adamganabe nyiro kara

⎐ٺون⎐
⊵SAAS⊴

⎐خيؤط▬الزعفران⎐
⊵AA75⊴

⎐شي▬ان⎐
⊵artm15⊴

⎐بلومينجوير⎐
⊵B39⊴
JmU
Hawardǝ: Nyiga wuzǝna

⎐ٺون⎐
⊵SAAS⊴

⎐اىؤا⎐
⊵ZZ900⊴
Jmu
Hawardǝ: Nyiga wuzǝna

⎐ٺون⎐
⊵SAAS⊴

⎐اىؤا⎐
⊵ZZ900⊴
Jmu


## Retrieval-Augmented Generation (RAG) 

For more about RAG, please read [Retrieval-Augmented Generation (RAG) with Atlas Vector Search](https://www.mongodb.com/docs/atlas/atlas-vector-search/rag/#std-label-avs-rag).

In [14]:
from openai import OpenAI

delimiter = '###'
chat_model = 'gpt-4o'
temperature = 0

chat_history = [{"role": "system", "content": """you are a chabot answer user questions based on the returned tweets"""}]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})
    
    tweets = vector_search(prompt)
    chat_history.append({"role": "system", "content": f"here the returned tweets delimitered by {delimiter}{tweets}{delimiter}"})

    response = client.chat.completions.create(
        model=chat_model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  food


Chatbot: The tweets that mention food include a user sharing about having two breakfasts aboard the "飛鳥II" ship and later enjoying a hot dog at さんばしや while overlooking the home port. No other tweets directly reference food-related topics.


You:  clothes


Chatbot: The returned tweets do not specifically mention clothing or related topics.


You:  sports


Chatbot: The tweets related to sports mention various events and achievements:

1. A LinkedIn Sun Belt scoreboard highlights JMU's dominance, with the school winning for the 7th week in a row.
2. The NACE playoffs are underway, featuring Rocket League teams from St Clair Academy and JMU, with matches scheduled against WVU.
3. App State had a successful weekend with a baseball series win at home over ODU, a softball series sweep at JMU, and a winning streak extended to 5 in tennis.
4. A tweet mentions a NASCAR-related event with participation from schools like JMU, UVA, Virginia Tech, and Liberty.
5. A tweet includes a mention of the JMU Dukes, likely referring to their sports teams.




You:  election


Chatbot: The tweets regarding elections cover various topics:

1. A tweet refers to an "election ploy," hinting at something possibly being used for political gain.
2. Multiple tweets describe the current election as "the most important election of our lifetimes."
3. One user mentions the convenience of the election period ending after election day.
4. There's an "Election for dummies" video encouraging votes for Donald Trump.
5. Another tweet questions the idea that "all politics are local" in the context of the current election.
6. A user emphasizes the importance of voter turnout to remove any doubt about the election winner.
7. A tweet expresses eagerness for the election to conclude.
8. A guide for undecided voters suggests avoiding candidates supported by Nazis.
9. Concerns are raised about fraudulent elections and the responsibility of clerks and canvassing boards.
10. A tweet discusses election night vote counting in Puerto Rico and mentions a shift to electronic voting with Do

You:  education


Chatbot: The tweets related to education include:

1. A user expressing that their key concerns with any election remain infrastructure, education, and healthcare, indicating the importance they place on these sectors.
2. Another tweet mentions learning about an election in middle school, highlighting the educational aspect of learning about governance and political processes. 

No other specific mentions of education are present in the tweets.


## Reference

- *“Introducing Text and Code Embeddings.”* n.d. OpenAI. Accessed October 31, 2024. https://openai.com/index/introducing-text-and-code-embeddings/.
- *“What Are Vector Databases?”* n.d. MongoDB. Accessed October 31, 2024. https://www.mongodb.com/resources/basics/databases/vector-databases.
